In [1]:
import sys
import os
import numpy as np
# from progressbar import progressbar as pb
import xarray as xr
import dask.dataframe as dd
import pandas as pd
from timeit import default_timer as timer


direc_path = "/data/lustre/parquet_ratio_complete"
store_path = "/data/lustre/parquet_ascent_tmp"

file_list = [os.path.join(direc_path, f) + "/*.parquet" for f in os.listdir(direc_path)]

window_conv_400 = 1 * 60 * 60
window_conv_600 = 3 * 60 * 60
window_slan_400 = 35 * 6 * 60 
window_slan_400_min = 15 * 6 * 60 
window_slan_600 = 22  * 60 * 60 
window_slan_600_min = 65 * 6 * 60 

def differ_400(x):
    # Pressure goes up -> trajectory goes down
    if x.argmax() > x.argmin():
        return False
    return ((x.max()-x.min()) >= 40000)

def differ_600(x):
    # Pressure goes up -> trajectory goes down
    if x.argmax() > x.argmin():
        return False
    return ((x.max()-x.min()) >= 60000)

# differ_400 = lambda x: ((x.max()-x.min()) >= 4000)
# differ_600 = lambda x: ((x.max()-x.min()) >= 6000)

def differ_slan_400(x):
    delta = differ_400(x)
    if not delta:
        return False
    smaller = pd.Series(x).rolling(window_slan_400_min, min_periods=1).apply(differ_400, raw=True).astype(dtype=bool).any()
    if smaller:
        return False
    return True

def differ_slan_600(x):
    delta = differ_600(x)
    if not delta:
        return False
    smaller = pd.Series(x).rolling(window_slan_600_min, min_periods=1).apply(differ_600, raw=True).astype(dtype=bool).any() 
    if smaller:
        return False
    return True

In [2]:
written = False
get_new = True


for i, f in enumerate(file_list):
    if get_new:
        print("Get new")
        t = timer()
        get_new = False
        df = dd.read_parquet(f)
        name = f.split("_ratio.parq")
        name = name[0]
        name = name.split("_t")
        name = name[-1] + "t"
        df["netcdf"] = name
        df["traj_index"] = i
#         t2 = timer()
#         print("Assigned netcdf and traj_index after {} s".format(t2-t))
        conv_400 = df["p"].rolling(window_conv_400, min_periods=1).apply(differ_400, raw=True).astype(dtype=bool)
#         conv_600 = df["p"].rolling(window_conv_600, min_periods=1).apply(differ_600, raw=True).astype(dtype=bool)
#         t4 = timer()
#         print("Conv done after {} s".format(t4-t2))
#         slan_400 = df["p"].rolling(window_slan_400, min_periods=1).apply(differ_slan_400, raw=True).astype(dtype=bool)
#         t5 = timer()
#         print("slan 400 done after {} s".format(t5-t4))
#         slan_600 = df["p"].rolling(window_slan_600, min_periods=1).apply(differ_slan_600, raw=True).astype(dtype=bool)
#         t3 = timer()
#         print("Apply done after {} s".format(t3-t2))
        df = df.assign(conv_400 = conv_400)
#         df = df.assign(conv_600 = conv_600)
#         df = df.assign(slan_400 = slan_400)
#         df = df.assign(slan_600 = slan_600)
#         t5 = timer()
#         print("Assigning conv and slan done after {} s".format(t5-t3))
        print("Name: {}, {}".format(name, i))
        print(df["netcdf"].unique())
    else:
        print("Get small set")
        small_set = dd.read_parquet(f)
        name = f.split("_ratio.parq")
        name = name[0]
        name = name.split("_t")
        name = name[-1] + "t"
        small_set["netcdf"] = name
        small_set["traj_index"] = i
        print("Got netcdf {} and traj_index {}".format(name, i))
        print(df["netcdf"].unique())
        conv_400 = small_set["p"].rolling(window_conv_400, min_periods=1).apply(differ_400, raw=True).astype(dtype=bool)
#         conv_600 = small_set["p"].rolling(window_conv_600, min_periods=1).apply(differ_600, raw=True).astype(dtype=bool)
#         slan_400 = small_set["p"].rolling(window_slan_400, min_periods=1).apply(differ_slan_400, raw=True).astype(dtype=bool)
#         slan_600 = small_set["p"].rolling(window_slan_600, min_periods=1).apply(differ_slan_600, raw=True).astype(dtype=bool)
        small_set = small_set.assign(conv_400 = conv_400)
#         small_set = small_set.assign(conv_600 = conv_600)
#         small_set = small_set.assign(slan_400 = slan_400)
#         small_set = small_set.assign(slan_600 = slan_600)
#         print("Assigned conv and slan")
        df = dd.concat([df, small_set])
#         print("Concat done")

    if (i%2==0 and i>0) or i == len(file_list)-1:
        if written:
            print("Attempt to append")
#             print(df.describe())
            dd.to_parquet(df, store_path, append=True, ignore_divisions=True, compression="snappy") # ignore_divisions=True,
            t2 = timer()
            print("Writing step {} done in {} s".format(i, t2-t))
            break
        else:
            print("Attempt to write")
#             print(df.describe())
            dd.to_parquet(df, store_path, compression="snappy")
            written = True
        t2 = timer()
        print("Writing step {} done in {} s".format(i, t2-t))
        get_new = True

Get new
Name: 003840_p001t, 0
Dask Series Structure:
npartitions=1
    object
       ...
Name: netcdf, dtype: object
Dask Name: unique-agg, 10 tasks
Get small set
Got netcdf 003360_p001t and traj_index 1
Dask Series Structure:
npartitions=1
    object
       ...
Name: netcdf, dtype: object
Dask Name: unique-agg, 10 tasks
Get small set
Got netcdf 003120_p001t and traj_index 2
Dask Series Structure:
npartitions=1
    object
       ...
Name: netcdf, dtype: object
Dask Name: unique-agg, 21 tasks
Attempt to write
Writing step 2 done in 402.79867046745494 s
Get new
Name: 003120_p001t, 3
Dask Series Structure:
npartitions=1
    object
       ...
Name: netcdf, dtype: object
Dask Name: unique-agg, 10 tasks
Get small set
Got netcdf 003240_p001t and traj_index 4
Dask Series Structure:
npartitions=1
    object
       ...
Name: netcdf, dtype: object
Dask Name: unique-agg, 10 tasks
Attempt to append
Writing step 4 done in 2326.8392280340195 s


In [4]:
df_new = dd.read_parquet(store_path)
print(df_new["traj_index"].unique().compute())
print(df_new["netcdf"].unique().compute())

0    0
1    3
Name: traj_index, dtype: int64
0    003840_p001t
1    003120_p001t
Name: netcdf, dtype: object


In [5]:
df_new.divisions

(None, None, None)

# Append test

In [1]:
import pandas as pd
from dask import dataframe as dd 
import numpy as np

df = pd.DataFrame({"A": np.random.randint(low=0, high=3, size=(50)),
                   "B": np.random.rand(50)})


In [2]:
df_2 = df[df.A == 1]
df_2

,A,B
0,1,0.927292
4,1,0.322716
5,1,0.833897
10,1,0.765410
14,1,0.845994
18,1,0.421380
21,1,0.043643
23,1,0.922619
25,1,0.468590
26,1,0.790923


In [3]:
ddf = dd.from_pandas(df, npartitions=1)
ddf = ddf.assign(C = False)
# ddf.compute()

In [4]:
ddf_2 = ddf[ddf.A == 1]
lf = lambda x: (x.max() < 0.75)
res = ddf_2["B"].rolling(3, min_periods=1).apply(lf, raw=True).astype(dtype=bool)

In [5]:
ddf_3 = ddf_2.assign(C = res)
ddf_3.compute()

,A,B,C
0,1,0.927292,False
4,1,0.322716,False
5,1,0.833897,False
10,1,0.765410,False
14,1,0.845994,False
18,1,0.421380,False
21,1,0.043643,False
23,1,0.922619,False
25,1,0.468590,False
26,1,0.790923,False


In [6]:
ddf_3 = ddf_3.repartition(npartitions=1)
ddf = ddf.merge(ddf_3, "outer")#.fillna(False)

In [7]:
ddf_2 = ddf[ddf.A == 0]
lf = lambda x: (x.max() < 0.75)
res = ddf_2["B"].rolling(3, min_periods=1).apply(lf, raw=True).astype(dtype=bool)
ddf_3 = ddf_2.assign(C = res)


In [8]:
ddf_3 = ddf_3.repartition(npartitions=1)
ddf = ddf.merge(ddf_3, "outer")#.fillna(False)

In [9]:
ddf.compute()

,A,B,C
0,1,0.927292,False
1,2,0.631862,False
2,0,0.269116,False
3,0,0.726272,False
4,1,0.322716,False
...,...,...,...
56,0,0.167053,True
57,0,0.278814,True
58,0,0.314422,True
59,0,0.389145,True


In [10]:
ddf_3.compute()

,A,B,C
2,0,0.269116,True
3,0,0.726272,True
6,0,0.167053,True
15,0,0.278814,True
16,0,0.936934,False
19,0,0.601019,False
20,0,0.517301,False
22,0,0.314422,True
27,0,0.389145,True
30,0,0.135445,True


In [11]:
res_c = res.compute()

In [13]:
res_c
res

Dask Series Structure:
npartitions=3
0     bool
17     ...
34     ...
49     ...
Name: B, dtype: bool
Dask Name: astype, 23 tasks